# **Part 1 - Business Problem**

### **Business Problem: Automated Nuclei Segmentation for Breast Cancer Diagnosis Support**


**Business Context**

Breast cancer remains one of the leading causes of cancer-related deaths worldwide, especially in aggressive subtypes like **Triple Negative Breast Cancer (TNBC)**. Accurate and timely analysis of histopathology slides is critical for effective diagnosis and treatment planning. However, manual annotation and segmentation of cell nuclei in Hematoxylin and Eosin (H\&E) stained slides is time-consuming, subjective, and prone to inter-pathologist variability.

**Business Challenge**

Hospitals, diagnostic labs, and cancer research centers face the following challenges:

* **Limited Pathologist Resources:** The global shortage of trained pathologists leads to diagnostic delays.
* **Annotation Bottleneck:** Manual segmentation of nuclei is labor-intensive and not scalable for large patient volumes.
* **Inter-observer Variability:** Manual interpretations vary significantly between pathologists, affecting diagnostic consistency.
* **Need for Quantitative Analysis:** Automated and consistent nuclear segmentation is essential for downstream tasks such as cancer grading, prognosis estimation, and treatment decision support.

**Business Objective**

The objective of this project is to develop a **Deep Learning-based automated nuclei segmentation system** for breast tumor histopathology images, with special focus on TNBC samples.

The solution aims to:

* Automatically detect and segment cell nuclei in H\&E-stained breast tumor images.
* Improve speed and accuracy compared to manual annotations.
* Support pathologists in making faster and more consistent diagnostic decisions.
* Enable large-scale analysis of patient samples for clinical and research applications.

**Expected Business Impact**

**Increased diagnostic efficiency**
**Reduced workload for pathologists**
**Improved accuracy and consistency in cancer diagnosis**
**Better patient outcomes through timely treatment decisions**

This project will serve as a critical step towards implementing AI-powered digital pathology solutions in real-world clinical settings.

# **Part 2 - API Kaggle**

In [ ]:
import zipfile
from google.colab import files

In [ ]:
files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

# **Part 3 - Baixando dataset**

In [ ]:
# Downloading database from kaggle
!kaggle datasets download tuanledinh/processedoriginalmonuseg

In [ ]:
# Target directory where the dataset will be extracted
target_dir = "/content/dataset/CNN/Breast"

In [ ]:
# Path to the zip file containing the Bitcoin tweets dataset
zf = "/content/processedoriginalmonuseg.zip"

In [ ]:
# Open the zip file containing the dataset
zfile = zipfile.ZipFile(zf)

In [ ]:
# Extract all contents of the zip file to the target directory
zfile.extractall(target_dir)

# **Part 4 - Importando as bibliotecas**

In [ ]:
##

#
import os
import cv2

#
import pandas as pd
import numpy as np

#
import seaborn as sns
import matplotlib.pyplot as plt

#
from PIL import Image

#
from glob import glob

#
from sklearn.model_selection import train_test_split

#
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout

#
from tensorflow.keras import backend as K
from tensorflow.keras.applications import VGG16

#
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve, average_precision_score

#
from tqdm import tqdm

# **Part 5 - Database**

In [ ]:
def load_images_and_masks(img_dir, mask_dir, img_size=(256, 256)):
    images = []
    masks = []

    img_files = sorted(os.listdir(img_dir))

    for file_name in img_files:
        img_path = os.path.join(img_dir, file_name)
        mask_path = os.path.join(mask_dir, file_name)

        # Carrega imagem
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        img = img / 255.0  # Normaliza para 0-1

        # Carrega máscara
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, img_size)
        mask = mask / 255.0  # Normaliza máscara binária (0-1)
        mask = np.expand_dims(mask, axis=-1)  # Formato (H, W, 1)

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

In [ ]:
# Caminhos das pastas

#
train_img_dir = "/content/dataset/CNN/Breast/processed-original-monuseg/train_folder/img"

#
train_mask_dir = "/content/dataset/CNN/Breast/processed-original-monuseg/train_folder/labelcol"

In [ ]:
def visualize_samples(X, Y, num_samples=6):
    plt.figure(figsize=(12, num_samples * 2))

    for i in range(num_samples):
        # Imagem original
        plt.subplot(num_samples, 2, 2*i+1)
        plt.imshow(X[i])
        plt.title(f'Input Image {i+1}')
        plt.axis('off')

        # Máscara correspondente
        plt.subplot(num_samples, 2, 2*i+2)
        plt.imshow(Y[i].squeeze(), cmap='gray')
        plt.title(f'Mask {i+1}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Carregar os dados
X_train, Y_train = load_images_and_masks(train_img_dir, train_mask_dir)

In [ ]:
# Visualizar
visualize_samples(X_train, Y_train, num_samples=6)

# **Rede Neural - UNET**

In [ ]:
def unet_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

#
def dice_coef(y_true, y_pred, smooth=1e-7):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

#
def iou_metric(y_true, y_pred, smooth=1e-7):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', iou_metric, dice_coef])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', iou_metric, dice_coef])

#
model.summary()

In [ ]:
# treinamento rede neural
history_unet = model.fit(X_train, Y_train,
                         epochs=120,
                         batch_size=8,
                         validation_split=0.2)

In [ ]:
def plot_training_history(history):
    # Plot Loss
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history_unet.history['loss'], label='Training Loss')
    plt.plot(history_unet.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss During Training')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(False)

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history_unet.history['accuracy'], label='Training Accuracy')
    plt.plot(history_unet.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy During Training')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(False)

    plt.tight_layout()
    plt.show()

# Visualizando desempenho da rede
plot_training_history(history_unet)

In [ ]:
#
def plot_iou_and_dice_training_seaborn(history):
    sns.set(style="whitegrid")  # Estilo de fundo

    plt.figure(figsize=(12, 5))

    # === IoU ===
    plt.subplot(1, 2, 1)
    sns.lineplot(x=range(len(history_unet.history['iou_metric'])), y=history_unet.history['iou_metric'], label='Training IoU', color='blue')
    sns.lineplot(x=range(len(history_unet.history['val_iou_metric'])), y=history_unet.history['val_iou_metric'], label='Validation IoU', color='orange')
    plt.title('IoU Metric During Training')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()
    plt.grid(False)
    plt.tight_layout()

    # === Dice ===
    plt.subplot(1, 2, 2)
    sns.lineplot(x=range(len(history_unet.history['dice_coef'])), y=history_unet.history['dice_coef'], label='Training Dice', color='green')
    sns.lineplot(x=range(len(history_unet.history['val_dice_coef'])), y=history_unet.history['val_dice_coef'], label='Validation Dice', color='red')
    plt.title('Dice Coefficient During Training')
    plt.xlabel('Epoch')
    plt.ylabel('Dice Coefficient')
    plt.legend()
    plt.tight_layout()
    plt.grid(False)
    plt.show()

# Exemplo de uso:
plot_iou_and_dice_training_seaborn(history_unet)

In [ ]:
# Pegando uma imagem de exemplo
i = 0
test_img = X_train[i]
ground_truth = Y_train[i]

In [ ]:
#
pred = model.predict(np.expand_dims(test_img, axis=0))[0]

In [ ]:
def plot_segmentation_overlay(test_img, ground_truth, pred_mask):
    plt.figure(figsize=(15,5))

    # Imagem original
    plt.subplot(1, 4, 1)
    plt.title('Input Image')
    plt.imshow(test_img)
    plt.axis('off')

    # Ground Truth
    plt.subplot(1, 4, 2)
    plt.title('Ground Truth Mask')
    plt.imshow(ground_truth.squeeze(), cmap='gray')
    plt.axis('off')

    # Predicted Mask
    plt.subplot(1, 4, 3)
    plt.title('Predicted Mask')
    plt.imshow(pred_mask.squeeze(), cmap='gray')
    plt.axis('off')

    # Overlay com Colormap (ex: Jet)
    plt.subplot(1, 4, 4)
    plt.title('Overlay Prediction')
    overlay = plt.imshow(test_img, alpha=0.7)
    plt.imshow(pred_mask.squeeze(), cmap='jet', alpha=0.3)  # Ajuste alpha como quiser
    plt.axis('off')

    plt.tight_layout()
    plt.show()

#
plot_segmentation_overlay(test_img, ground_truth, pred)

In [ ]:
#
pred_binary = (pred > 0.5).astype(np.uint8)
plot_segmentation_overlay(test_img, ground_truth, pred_binary)

In [ ]:
def plot_and_save_predictions(X, Y_true, Y_pred, num_samples=6, save_path="segmentation_results.png", threshold=0.5, cmap_overlay='plasma'):
    plt.figure(figsize=(16, num_samples * 3))

    for i in range(num_samples):
        # Binariza a predição
        pred_thresh = (Y_pred[i] > threshold).astype(np.float32)

        # Calcula métricas
        iou = calculate_iou(Y_true[i], pred_thresh)
        dice = calculate_dice(Y_true[i], pred_thresh)

        # Input Image
        plt.subplot(num_samples, 4, 4*i + 1)
        plt.imshow(X[i])
        plt.title(f"Input {i+1}")
        plt.axis('off')

        # Ground Truth Mask
        plt.subplot(num_samples, 4, 4*i + 2)
        plt.imshow(Y_true[i].squeeze(), cmap='gray')
        plt.title(f"Ground Truth {i+1}")
        plt.axis('off')

        # Predicted Mask (Thresholded)
        plt.subplot(num_samples, 4, 4*i + 3)
        im_pred = plt.imshow(pred_thresh.squeeze(), cmap='gray')
        plt.title(f"Predicted {i+1}\nIoU: {iou:.2f} | Dice: {dice:.2f}")
        plt.axis('off')

        # Overlay (Input + Probabilidade com Colorbar)
        plt.subplot(num_samples, 4, 4*i + 4)
        plt.imshow(X[i])
        im_overlay = plt.imshow(Y_pred[i].squeeze(), cmap=cmap_overlay, alpha=0.5)
        plt.title(f"Overlay {i+1}")

        # Colorbar apenas para overlay
        cbar = plt.colorbar(im_overlay, fraction=0.046, pad=0.04)
        cbar.set_label('Predicted Probability', rotation=270, labelpad=15)
        plt.axis('off')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.show()
    print(f"\n Resultado salvo em: {save_path}")

#
def calculate_iou(y_true, y_pred, smooth=1e-7):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

#
def calculate_dice(y_true, y_pred, smooth=1e-7):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

In [ ]:
# Previsão modelo
num_samples = 6
X_test = X_train[:num_samples]
Y_test = Y_train[:num_samples]
Y_pred = model.predict(X_test)

In [ ]:
# Previsão
plot_and_save_predictions(X_test, Y_test, Y_pred, num_samples=num_samples, save_path="nuclei_segmentation_grid.png", threshold=0.5, cmap_overlay='plasma')

# Salvando imagem previsão
plt.tight_layout()
plt.savefig("breast_nuclei_segmentation_results.png", dpi=300)
plt.show()

In [ ]:
# Salvando rede
model.save("unet_UNet_breast_segmentation.h5")

# **Metricas e avaliação**

In [ ]:
# Exemplo usando um batch de teste
num_samples = 10
X_test = X_train[:num_samples]
Y_test = Y_train[:num_samples]

# Faz a predição
Y_pred = model.predict(X_test)

# Binariza a predição
Y_pred_binary = (Y_pred > 0.3).astype(np.uint8)

# Binariza também o Ground Truth (se tiver ruídos ou for float)
Y_test_binary = (Y_test > 0.3).astype(np.uint8)

# Flatten ambos para 1D
y_true_flat = Y_test_binary.flatten()
y_pred_flat = Y_pred_binary.flatten()

# Classification report
report = classification_report(y_true_flat, y_pred_flat, target_names=["Background", "Nuclei"])
print(report)

In [ ]:
# Faz a predição nas amostras de teste
num_samples = 10
X_test = X_train[:num_samples]
Y_test = Y_train[:num_samples]

Y_pred = model.predict(X_test)

# Ground Truth binário
Y_test_binary = (Y_test > 0.3).astype(np.uint8)

# Flatten para 1D
y_true_flat = Y_test_binary.flatten()
y_pred_flat = Y_pred.flatten()  # Aqui usamos a probabilidade contínua (antes do threshold)

# Calcular a curva ROC e AUC
fpr, tpr, thresholds = roc_curve(y_true_flat, y_pred_flat)
roc_auc = auc(fpr, tpr)

# Plotar a Curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Pixel-wise Segmentation')
plt.legend(loc="lower right")
plt.grid(False)
plt.show()

In [ ]:
# Cálculo
precision, recall, thresholds_pr = precision_recall_curve(y_true_flat, y_pred_flat)
average_precision = average_precision_score(y_true_flat, y_pred_flat)

# Plot
plt.figure(figsize=(8,6))
plt.plot(recall, precision, color='purple', lw=2, label=f'AP = {average_precision:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Pixel-wise Segmentation')
plt.legend(loc='lower left')
plt.grid(True)
plt.show()


# **Modelo 2 - Rede Neural**

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0, MobileNetV2
import numpy as np

# ===== Métricas personalizadas =====
def iou_metric(y_true, y_pred, smooth=1e-7):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def dice_coef(y_true, y_pred, smooth=1e-7):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

# ===== U-Net Básico =====
def unet_model(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# ===== Função para U-Net com backbone =====
def build_unet_with_backbone(base_model, skip_layers, input_size=(256, 256, 3)):
    inputs = base_model.input
    x = base_model.output
    skips = [base_model.get_layer(name).output for name in skip_layers][::-1]

    for skip in skips:
        while x.shape[1] < skip.shape[1] or x.shape[2] < skip.shape[2]:
            x = Conv2DTranspose(skip.shape[-1], (2, 2), strides=(2, 2), padding='same')(x)
        x = concatenate([x, skip])
        x = Conv2D(skip.shape[-1], (3, 3), activation='relu', padding='same')(x)
        x = Conv2D(skip.shape[-1], (3, 3), activation='relu', padding='same')(x)

    while x.shape[1] < input_size[0] or x.shape[2] < input_size[1]:
        x = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)
    return Model(inputs, outputs)

# ===== Modelos =====
def unet_baseline(input_size=(256, 256, 3)):
    return unet_model(input_size)

def unet_vgg16(input_size=(256, 256, 3)):
    base = VGG16(weights='imagenet', include_top=False, input_shape=input_size)
    skip = ["block1_conv2", "block2_conv2", "block3_conv3", "block4_conv3"]
    return build_unet_with_backbone(base, skip, input_size)

def unet_resnet50(input_size=(256, 256, 3)):
    base = ResNet50(weights='imagenet', include_top=False, input_shape=input_size)
    skip = ["conv1_relu", "conv2_block3_out", "conv3_block4_out", "conv4_block6_out"]
    return build_unet_with_backbone(base, skip, input_size)

def unet_efficientnetb0(input_size=(256, 256, 3)):
    base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_size)
    skip = ["block2a_activation", "block3a_activation", "block4a_activation", "block6a_activation"]
    return build_unet_with_backbone(base, skip, input_size)

def unet_mobilenetv2(input_size=(256, 256, 3)):
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_size)
    skip = ["block_1_expand_relu", "block_3_expand_relu", "block_6_expand_relu", "block_13_expand_relu"]
    return build_unet_with_backbone(base, skip, input_size)

# ===== Lista de modelos (agora depois das definições) =====
model_functions = [("U-Net", unet_baseline),("VGG16", unet_vgg16),("ResNet50", unet_resnet50),("EfficientNetB0", unet_efficientnetb0),("MobileNetV2", unet_mobilenetv2)]

# ===== Loop para mostrar o summary de cada modelo =====
for model_name, model_fn in model_functions:
    print(f"\n============================")
    print(f"MODEL SUMMARY: {model_name}")
    print(f"============================\n")
    model = model_fn()
    model.summary()

In [ ]:
%%time

from tensorflow.keras.optimizers import Adam

# ===== Dicionário para guardar os histories =====
history_dict = {}

# ===== Treinamento =====
for model_name, model_fn in model_functions:
    print(f"\n{'='*50}")
    print(f"INICIANDO TREINAMENTO: {model_name}")
    print(f"{'='*50}\n")

    # Cria o modelo
    model = model_fn()

    # Congelando backbone
    print()
    print(f"Freezing backbone para o modelo {model_name}...")
    print()
    for layer in model.layers[:20]:
        layer.trainable = False

    # Compilar e treinar com backbone congelado
    print()
    print(f"Treinando {model_name} com backbone congelado (fase 1)...")
    print()

    #
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    #
    history_frozen = model.fit(X_train, Y_train, epochs=20, batch_size=8, validation_split=0.2, verbose=1)

    # Liberar todas as camadas para fine-tuning
    print()
    print(f"Fine-tuning (liberando todas as camadas) para o modelo {model_name}...")
    print()

    #
    for layer in model.layers:
        layer.trainable = True

    # Compilar e treinar com learning rate menor
    print()
    print(f"Treinando {model_name} (fase fine-tuning)...")
    print()

    #
    model.compile(optimizer=Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    #
    history_finetune = model.fit(X_train, Y_train, epochs=30, batch_size=8, validation_split=0.2, verbose=1)

    # Unir histories (fase1 + fine-tuning)
    full_history = history_frozen
    for key in history_finetune.history.keys():
        full_history.history[key] = history_frozen.history.get(key, []) + history_finetune.history[key]

    # Salva no dicionário
    history_dict[model_name] = full_history

    print(f"\nFinalizado o treinamento de: {model_name}\n")

print("\nTreinamento completo para todos os modelos!")


In [ ]:
#
import os

#
os.makedirs('saved_models', exist_ok=True)
os.makedirs('weights', exist_ok=True)

#
for model_name, model_fn in model_functions:
    print(f"Salvando o modelo: {model_name}")

    # Recomendo recriar o modelo antes de carregar os pesos finais (se quiser, mas não obrigatório)
    model = model_fn()

    # Treinar novamente se necessário ou carregar os pesos mais recentes
    # Aqui assumindo que você já treinou e tem a variável `model` pronta ao final de cada loop.

    # Caminho de saída
    save_path = f"saved_models/{model_name}_breast_segmentation.keras"

    # Salvar o modelo inteiro (estrutura + pesos + optimizer)
    print()
    model.save(save_path)
    print(f"Modelo salvo em: {save_path}")

In [ ]:
# Isso vai listar os arquivos na pasta atual, incluindo os .h5
print(os.listdir())

In [ ]:
def plot_model_performance(history_dict):
    plt.figure(figsize=(14, 6))

    # ======= Loss =======
    plt.subplot(1, 2, 1)
    for model_name, history in history_dict.items():
        plt.plot(history.history['loss'], label=f'{model_name} - Train Loss')
        plt.plot(history.history['val_loss'], linestyle='--', label=f'{model_name} - Val Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(False)

    # ======= Accuracy =======
    plt.subplot(1, 2, 2)
    for model_name, history in history_dict.items():
        plt.plot(history.history['accuracy'], label=f'{model_name} - Train Acc')
        plt.plot(history.history['val_accuracy'], linestyle='--', label=f'{model_name} - Val Acc')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(False)
        plt.tight_layout()
        plt.show()

# Plot
plot_model_performance(history_dict)

In [ ]:
def calculate_iou_dice(y_true, y_pred_bin):
    intersection = np.sum(y_true * y_pred_bin)
    union = np.sum(y_true) + np.sum(y_pred_bin) - intersection
    iou = intersection / (union + 1e-7)
    dice = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred_bin) + 1e-7)
    return iou, dice

def plot_model_predictions(model_name, model, X_test, Y_test, threshold=0.5, num_samples=6):
    Y_pred = model.predict(X_test)

    plt.figure(figsize=(16, num_samples * 3))
    for i in range(num_samples):
        # Binariza a predição
        pred_bin = (Y_pred[i] > threshold).astype(np.float32)

        # Calcula IoU e Dice
        iou, dice = calculate_iou_dice(Y_test[i].flatten(), pred_bin.flatten())

        # Input
        plt.subplot(num_samples, 4, 4*i + 1)
        plt.imshow(X_test[i])
        plt.title(f"Input {i+1}")
        plt.axis('off')

        # Ground Truth
        plt.subplot(num_samples, 4, 4*i + 2)
        plt.imshow(Y_test[i].squeeze(), cmap='gray')
        plt.title(f"Ground Truth {i+1}")
        plt.axis('off')

        # Predicted Binary Mask
        plt.subplot(num_samples, 4, 4*i + 3)
        plt.imshow(pred_bin.squeeze(), cmap='gray')
        plt.title(f"Pred {i+1} | IoU: {iou:.2f} | Dice: {dice:.2f}")
        plt.axis('off')

        # Overlay (probabilidade original com colormap)
        plt.subplot(num_samples, 4, 4*i + 4)
        plt.imshow(X_test[i])
        im_overlay = plt.imshow(Y_pred[i].squeeze(), cmap='plasma', alpha=0.5)
        plt.title(f"Overlay {i+1}")
        plt.colorbar(im_overlay, fraction=0.046, pad=0.04)
        plt.axis('off')

    plt.suptitle(f"Predictions for {model_name}", fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
from tensorflow.keras.models import load_model

#
model_base_path = '/content/saved_models'

#
for model_name, model_fn in model_functions:
    print(f"\n============================")
    print(f"Generating predictions for: {model_name}")
    print(f"============================\n")

    # Nome correto do arquivo .keras
    keras_file = os.path.join(model_base_path, f"{model_name}_breast_segmentation.keras")

    # Corrige se houver nome com hífen (caso do U-Net)
    keras_file = keras_file.replace('U-Net', 'U-Net')  # Apenas por garantia

    if os.path.exists(keras_file):
        try:
            model = load_model(keras_file, compile=False)
            print(f"✅ Modelo carregado de: {keras_file}")
        except Exception as e:
            print(f"⚠️ Erro ao carregar o modelo: {e}")
            continue
    else:
        print(f"❌ Arquivo de modelo não encontrado: {keras_file}")
        continue

    # Faz as previsões e plota
    plot_model_predictions(model_name, model, X_test, Y_test, threshold=0.5, num_samples=num_samples)

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
import numpy as np
import os

def calculate_iou_dice(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    iou = intersection / (union + 1e-7)
    dice = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred) + 1e-7)
    return iou, dice

# Test set
num_samples = 10
X_test = X_train[:num_samples]
Y_test = Y_train[:num_samples]

# Binariza o Ground Truth
Y_test_binary = (Y_test > 0.5).astype(np.uint8)
y_true_flat = Y_test_binary.flatten()

model_base_path = "/content/saved_models"

for model_name, model_fn in model_functions:
    print(f"\n============================")
    print(f"Evaluation for: {model_name}")
    print(f"============================\n")

    keras_file = os.path.join(model_base_path, f"{model_name}_breast_segmentation.keras")

    if os.path.exists(keras_file):
        try:
            # Carregando o modelo completo
            model = load_model(keras_file, compile=False)
            print(f"✅ Modelo carregado de: {keras_file}")
        except Exception as e:
            print(f"⚠️ Erro ao carregar o modelo: {e}")
            continue
    else:
        print(f"❌ Arquivo não encontrado: {keras_file}")
        continue

    # Predição
    Y_pred = model.predict(X_test)
    Y_pred_binary = (Y_pred > 0.5).astype(np.uint8)
    y_pred_flat = Y_pred_binary.flatten()

    # Classification Report
    report = classification_report(y_true_flat, y_pred_flat, target_names=["Background", "Nuclei"])
    print(report)

    # IoU e Dice global
    iou, dice = calculate_iou_dice(y_true_flat, y_pred_flat)
    print(f"IoU Médio: {iou:.4f}")
    print(f"Dice Médio: {dice:.4f}")

In [ ]:
##

#
num_samples = 10
X_test = X_train[:num_samples]
Y_test = Y_train[:num_samples]
Y_test_binary = (Y_test > 0.5).astype(np.uint8)
y_true_flat = Y_test_binary.flatten()


#
plt.figure(figsize=(10, 8))

#
for model_name, model_fn in model_functions:
    print(f"\n============================")
    print(f"ROC Curve for: {model_name}")
    print(f"============================\n")

    #
    model = model_fn()

    # Corrigir nome do arquivo de pesos
    if model_name == "/content/unet_UNet_breast_segmentation.h5":
        weight_file = "/content/unet_UNet_breast_segmentation.h5"
    else:
        clean_name = model_name.replace('U-Net + ', '').replace(' ', '')
        weight_file = f"unet_{clean_name}_breast_segmentation.h5"

    try:
        model.load_weights(weight_file)
        print(f"Pesos carregados de: {weight_file}")
    except:
        print(f"Arquivo {weight_file} não encontrado. Pulando...")
        continue

    # Faz predição com as probabilidades (sem threshold)
    Y_pred = model.predict(X_test)
    y_pred_flat = Y_pred.flatten()

    # Calcula curva ROC
    fpr, tpr, thresholds = roc_curve(y_true_flat, y_pred_flat)
    roc_auc = auc(fpr, tpr)

    # Plot
    plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.4f})')

# Linha diagonal (baseline)
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Pixel-wise Segmentation (All Models)')
plt.legend(loc='lower right')
plt.grid(False)
plt.tight_layout()
plt.show()

# **Resultados Redes Neurais**

In [ ]:
# ===== Geração de DataFrame com Resultados =====

#
results = []

#
for model_name, history in history_dict.items():
    min_loss = min(history.history['val_loss'])
    max_acc = max(history.history['val_accuracy'])

    #
    result = {'Model': model_name,
              'Best Val Loss': round(min_loss, 4),
              'Best Val Accuracy': round(max_acc, 4)}

    # Se tiver IoU e Dice no history:
    if 'val_iou_metric' in history.history and 'val_dice_coef' in history.history:
        result['Best Val IoU'] = round(max(history.history['val_iou_metric']), 4)
        result['Best Val Dice'] = round(max(history.history['val_dice_coef']), 4)

    #
    results.append(result)

#
df_results = pd.DataFrame(results)

#
df_results = df_results.sort_values(by='Best Val Accuracy', ascending=False).reset_index(drop=True)

#
df_results

In [ ]:
def calculate_iou_dice(y_true, y_pred_bin):
    intersection = np.sum(y_true * y_pred_bin)
    union = np.sum(y_true) + np.sum(y_pred_bin) - intersection
    iou = intersection / (union + 1e-7)
    dice = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred_bin) + 1e-7)
    return iou, dice

# Test set
num_samples = 10
X_test = X_train[:num_samples]
Y_test = Y_train[:num_samples]
Y_test_binary = (Y_test > 0.5).astype(np.uint8)
y_true_flat = Y_test_binary.flatten()

# Lista de resultados
final_results = []

# Caminho onde estão os modelos .keras
model_base_path = "/content/saved_models"

for model_name, model_fn in model_functions:
    print(f"\n========== Avaliando modelo: {model_name} ==========\n")

    keras_file = os.path.join(model_base_path, f"{model_name}_breast_segmentation.keras")

    if not os.path.exists(keras_file):
        print(f"❌ Modelo não encontrado: {keras_file}")
        continue

    try:
        # Carregar o modelo completo
        model = load_model(keras_file, compile=False)
        print(f"✅ Modelo carregado: {keras_file}")

        # Predição
        Y_pred = model.predict(X_test)
        Y_pred_binary = (Y_pred > 0.5).astype(np.uint8)
        y_pred_flat = Y_pred_binary.flatten()

        # Classification report
        report_dict = classification_report(y_true_flat, y_pred_flat, target_names=["Background", "Nuclei"], output_dict=True)
        iou, dice = calculate_iou_dice(y_true_flat, y_pred_flat)

        result = {
            'Model': model_name,
            'Accuracy': round(report_dict['accuracy'], 4),
            'Precision (Nuclei)': round(report_dict['Nuclei']['precision'], 4),
            'Recall (Nuclei)': round(report_dict['Nuclei']['recall'], 4),
            'F1-Score (Nuclei)': round(report_dict['Nuclei']['f1-score'], 4),
            'Support (Nuclei)': int(report_dict['Nuclei']['support']),
            'Macro Avg F1': round(report_dict['macro avg']['f1-score'], 4),
            'Weighted Avg F1': round(report_dict['weighted avg']['f1-score'], 4),
            'IoU Médio': round(iou, 4),
            'Dice Médio': round(dice, 4)
        }

        final_results.append(result)

    except Exception as e:
        print(f"⚠️ Erro ao avaliar o modelo {model_name}: {e}")
        continue

# Resultado final como DataFrame
df_metrics = pd.DataFrame(final_results)
df_metrics = df_metrics.sort_values(by='Dice Médio', ascending=False).reset_index(drop=True)
df_metrics